In [1]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

seed = 28
torch.manual_seed(seed)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


### Load model to check

In [2]:
# from transformers import AutoTokenizer, AutoModelForCausalLM


# model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [3]:
# messages = [{"role": "user", "content": "What is gravity?"}]
# input_text=tokenizer.apply_chat_template(messages, tokenize=False)

# inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
# outputs = model.generate(inputs, max_new_tokens=50, temperature=0.2, top_p=0.9, do_sample=True)
# print(tokenizer.decode(outputs[0]))

### Load and prepare dataset

In [4]:
from datasets import load_dataset

dataset = load_dataset("esfrankel17/HelpSteer2_binarized", split='average_rating_split')

m:\python_projects\REINFORCE_implementation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dataset

Dataset({
    features: ['prompt', 'chosen', 'chosen_rating', 'rejected', 'rejected_rating'],
    num_rows: 8678
})

In [6]:
dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=seed)

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'chosen_rating', 'rejected', 'rejected_rating'],
        num_rows: 6942
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'chosen_rating', 'rejected', 'rejected_rating'],
        num_rows: 1736
    })
})

In [8]:
dataset["train"][0]

{'prompt': 'What is the point group of m3m? any crystal group get this symmetry? Since it is a cubic point group, referred to Oh? is C4 also a cubic point group?\n<extra_id_1>Assistant\nThe point group of m3m is m3m, which has order 16 and is a cubic point group. The crystal structure of m3m is cubic, and the symmetry of the m3m group is referred to as Oh. The point group C4 also has order 16 and is a cubic point group, and has a cubic crystal structure with symmetry referred to as Ci.\n<extra_id_1>User\nare m3m and C4 describing the same structure?\n<extra_id_1>Assistant\nYes, m3m and C4 describe the same crystal structure. This structure has a four-fold rotation axis along the center of the cube, and eight mirror planes along the diagonal directions of the cube.\n<extra_id_1>User\nGenerate a picture for me ?',
 'chosen': [{'content': 'What is the point group of m3m? any crystal group get this symmetry? Since it is a cubic point group, referred to Oh? is C4 also a cubic point group?\n

In [9]:
# check if there is more than one request-response pair
for example in dataset["train"]:
    if len(example["chosen"]) > 2:
        print(example["chosen"])
        break
    if len(example["rejected"]) > 2:
        print(example["rejected"])
        break

In [10]:
import numpy as np

all_rewards = np.array(
    dataset["train"]["chosen_rating"] + dataset["train"]["rejected_rating"]
)

mean_reward = np.mean(all_rewards)
std_reward = np.std(all_rewards)

print(f"mean = {mean_reward}\nstd = {std_reward}")

mean = 2.6000432152117545
std = 0.6627274849063368


In [11]:
def process_example(example):
    chosen_text = " ".join([msg["content"] for msg in example["chosen"]])
    rejected_text = " ".join([msg["content"] for msg in example["rejected"]])
    margin = (example["chosen_rating"] - example["rejected_rating"])
    
    return {"chosen": chosen_text, "rejected": rejected_text, "margin": margin}

In [12]:
processed_dataset = dataset.map(process_example)

In [13]:
processed_dataset["train"][0]

{'prompt': 'What is the point group of m3m? any crystal group get this symmetry? Since it is a cubic point group, referred to Oh? is C4 also a cubic point group?\n<extra_id_1>Assistant\nThe point group of m3m is m3m, which has order 16 and is a cubic point group. The crystal structure of m3m is cubic, and the symmetry of the m3m group is referred to as Oh. The point group C4 also has order 16 and is a cubic point group, and has a cubic crystal structure with symmetry referred to as Ci.\n<extra_id_1>User\nare m3m and C4 describing the same structure?\n<extra_id_1>Assistant\nYes, m3m and C4 describe the same crystal structure. This structure has a four-fold rotation axis along the center of the cube, and eight mirror planes along the diagonal directions of the cube.\n<extra_id_1>User\nGenerate a picture for me ?',
 'chosen': "What is the point group of m3m? any crystal group get this symmetry? Since it is a cubic point group, referred to Oh? is C4 also a cubic point group?\n<extra_id_1>A

In [14]:
columns_to_keep = [
    "chosen",
    "rejected",
    "margin"
]

processed_dataset["train"] = processed_dataset["train"].remove_columns(
    [col for col in processed_dataset["train"].column_names if col not in columns_to_keep]
)

processed_dataset["test"] = processed_dataset["test"].remove_columns(
    [col for col in processed_dataset["test"].column_names if col not in columns_to_keep]
)

In [15]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'margin'],
        num_rows: 6942
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'margin'],
        num_rows: 1736
    })
})

### Training Reward model with discrete scores using LoRa and RewardTrainer

In [16]:
from transformers import AutoTokenizer


model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [17]:
import os
import shutil

directory_path = r'./RM_output'

if os.path.exists(directory_path) and os.path.isdir(directory_path):
    shutil.rmtree(directory_path)
    print(f"Directory '{directory_path}' has been deleted.")
else:
    print(f"Directory '{directory_path}' does not exist.")

Directory './RM_output' has been deleted.


In [18]:
from peft import LoraConfig, TaskType
from transformers import AutoModelForSequenceClassification
from trl import RewardTrainer, RewardConfig


model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
finetune_name = "SmolLM2-135M-Instruct-Reward-probabilistic"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=10,
    problem_type="multi_label_classification",
).to(device)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-135M-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
import torch.nn as nn
import torch.nn.functional as F


def custom_reward_loss(logits_w, logits_l, eps=1e-8):
    p_w = F.softmax(logits_w, dim=-1)
    p_l = F.softmax(logits_l, dim=-1)

    prod = p_w.unsqueeze(2) * p_l.unsqueeze(1)
    mask = torch.tril(torch.ones((10, 10), device=prod.device), diagonal=-1)
    p = (prod * mask).sum(dim=(-1, -2))

    loss = -torch.log(p + eps)
    return loss.mean()


class ImprovedRewardTrainer(RewardTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs_w = model(
            input_ids=inputs["input_ids_chosen"],
            attention_mask=inputs["attention_mask_chosen"]
        )
        outputs_l = model(
            input_ids=inputs["input_ids_rejected"],
            attention_mask=inputs["attention_mask_rejected"]
        )
        
        logits_w = outputs_w.logits
        logits_l = outputs_l.logits

        loss = custom_reward_loss(logits_w, logits_l)
        return (loss, outputs_w) if return_outputs else loss
    
    def compute_metrics(self, eval_pred, **kwargs):
        return {}

    def prediction_step(self, model, inputs, prediction_loss_only=False, ignore_keys=None):
        with torch.no_grad():
            outputs_w = model(
                input_ids=inputs["input_ids_chosen"],
                attention_mask=inputs["attention_mask_chosen"]
            )
            outputs_l = model(
                input_ids=inputs["input_ids_rejected"],
                attention_mask=inputs["attention_mask_rejected"]
            )

        rewards_w = (F.softmax(outputs_w.logits, dim=-1) * torch.arange(1, 11, device=model.device)).sum(dim=-1)
        rewards_l = (F.softmax(outputs_l.logits, dim=-1) * torch.arange(1, 11, device=model.device)).sum(dim=-1)
        
        logits = torch.stack([rewards_w, rewards_l], dim=1)
        loss = custom_reward_loss(outputs_w.logits, outputs_l.logits)
        
        return (loss, logits, None) if not prediction_loss_only else (loss, None, None)

In [21]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.2,
)

training_args = RewardConfig(
    output_dir="./RM_output",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    eval_accumulation_steps=4,
    learning_rate=5e-5,
    max_length=512,
    fp16=True,
    bf16=False,
    tf32=False,
    logging_steps=25,
    eval_strategy="steps",
    eval_steps=25,
    center_rewards_coefficient=0.01,  # encouraging the model to produce mean-zero outputs
    report_to=["tensorboard"],
    logging_dir="./RM_output/logs",
    hub_model_id=finetune_name,
)

trainer = ImprovedRewardTrainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    peft_config=peft_config,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
25,1.061700,0.995526
50,0.969200,0.928911
75,0.918500,0.882135
100,0.875000,0.862527
125,0.874000,0.850927
150,0.857700,0.839825
175,0.826900,0.831028
200,0.827900,0.825378
225,0.835600,0.821981
250,0.806500,0.819056


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Then I saw another mighty angel coming down   │ Then I saw another mighty angel coming down  │ [7.4955, 7.1097] │
│ from heaven, wrapped in a cloud, with a       │ from heaven, wrapped in a cloud, with a      │                  │
│ rainbow over his head, and his face was like  │ rainbow over his head, and his face was like │                  │
│ the sun, and his legs like pillars of fire.   │ the sun, and his legs like pillars of fire.  │                  │
│ He had a little scroll open in his hand. And  │ He had a little scroll open in his hand. And │                  │
│ he set his right foot on the sea, and his     │ he set his right foot on the sea, and his    │                  │
│ left foot on the land, and called out with a  │ left foot on the land, and called out with a │                  │
│ loud voice, like a lion roaring. When he      │ loud voice, like a lion roaring. When he     │                  │
│ called out, the seven thunders sounded. And   │ called out, the seven thunders sounded. And  │                  │
│ when the seven thunders had sounded, I was    │ when the seven thunders had sounded, I was   │                  │
│ about to write, but I heard a voice from      │ about to write, but I heard a voice from     │                  │
│ heaven saying, “Seal up what the seven        │ heaven saying, “Seal up what the seven       │                  │
│ thunders have said, and do not write it       │ thunders have said, and do not write it      │                  │
│ down.” And the angel whom I saw standing on   │ down.” And the angel whom I saw standing on  │                  │
│ the sea and on the land raised his right hand │ the sea and on the land raised his right     │                  │
│ to heaven This is a quote from the Book of    │ hand to heaven And he swore by him who lives │                  │
│ Revelation in the Bible. The passage          │ forever and ever, who created heaven and     │                  │
│ describes an angel with a rainbow over his    │ what is in it, the earth and what is in it,  │                  │
│ head, a face like the sun, and legs like      │ and the sea and what is in it, that there    │                  │
│ pillars of fire. The angel has a small scroll │ would be no more delay, but that in the days │                  │
│ in his hand and sets one foot on the sea and  │ of the sixth seal, there would be a great    │                  │
│ the other on the land. The angel then roars   │ earthquake, and the sun became black as      │                  │
│ like a lion and the seven thunders sound. The │ sackcloth made of goat hair, the whole moon  │                  │
│ voice from heaven tells the angel to seal up  │ became like blood, and the stars of the sky  │                  │
│ what the seven thunders have said and not to  │ fell to the earth as the fig tree sheds its  │                  │
│ write it down.                                │ winter fruit when shaken by the wind. The    │                  │
│                                               │ sky vanished like a scroll that is being     │                  │
│                                               │ rolled up, and every mountain and island was │                  │
│                                               │ removed from its place. Then the kings of    │                  │
│                                               │ the earth and the great ones and the         │                  │
│                                               │ generals and the rich and the powerful, and  │                  │
│                                               │ everyo

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Then I saw another mighty angel coming down   │ Then I saw another mighty angel coming down  │ [7.753, 7.0899]  │
│ from heaven, wrapped in a cloud, with a       │ from heaven, wrapped in a cloud, with a      │                  │
│ rainbow over his head, and his face was like  │ rainbow over his head, and his face was like │                  │
│ the sun, and his legs like pillars of fire.   │ the sun, and his legs like pillars of fire.  │                  │
│ He had a little scroll open in his hand. And  │ He had a little scroll open in his hand. And │                  │
│ he set his right foot on the sea, and his     │ he set his right foot on the sea, and his    │                  │
│ left foot on the land, and called out with a  │ left foot on the land, and called out with a │                  │
│ loud voice, like a lion roaring. When he      │ loud voice, like a lion roaring. When he     │                  │
│ called out, the seven thunders sounded. And   │ called out, the seven thunders sounded. And  │                  │
│ when the seven thunders had sounded, I was    │ when the seven thunders had sounded, I was   │                  │
│ about to write, but I heard a voice from      │ about to write, but I heard a voice from     │                  │
│ heaven saying, “Seal up what the seven        │ heaven saying, “Seal up what the seven       │                  │
│ thunders have said, and do not write it       │ thunders have said, and do not write it      │                  │
│ down.” And the angel whom I saw standing on   │ down.” And the angel whom I saw standing on  │                  │
│ the sea and on the land raised his right hand │ the sea and on the land raised his right     │                  │
│ to heaven This is a quote from the Book of    │ hand to heaven And he swore by him who lives │                  │
│ Revelation in the Bible. The passage          │ forever and ever, who created heaven and     │                  │
│ describes an angel with a rainbow over his    │ what is in it, the earth and what is in it,  │                  │
│ head, a face like the sun, and legs like      │ and the sea and what is in it, that there    │                  │
│ pillars of fire. The angel has a small scroll │ would be no more delay, but that in the days │                  │
│ in his hand and sets one foot on the sea and  │ of the sixth seal, there would be a great    │                  │
│ the other on the land. The angel then roars   │ earthquake, and the sun became black as      │                  │
│ like a lion and the seven thunders sound. The │ sackcloth made of goat hair, the whole moon  │                  │
│ voice from heaven tells the angel to seal up  │ became like blood, and the stars of the sky  │                  │
│ what the seven thunders have said and not to  │ fell to the earth as the fig tree sheds its  │                  │
│ write it down.                                │ winter fruit when shaken by the wind. The    │                  │
│                                               │ sky vanished like a scroll that is being     │                  │
│                                               │ rolled up, and every mountain and island was │                  │
│                                               │ removed from its place. Then the kings of    │                  │
│                                               │ the earth and the great ones and the         │                  │
│                                               │ generals and the rich and the powerful, and  │                  │
│                                               │ everyo

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Then I saw another mighty angel coming down   │ Then I saw another mighty angel coming down  │ [7.738, 6.983]   │
│ from heaven, wrapped in a cloud, with a       │ from heaven, wrapped in a cloud, with a      │                  │
│ rainbow over his head, and his face was like  │ rainbow over his head, and his face was like │                  │
│ the sun, and his legs like pillars of fire.   │ the sun, and his legs like pillars of fire.  │                  │
│ He had a little scroll open in his hand. And  │ He had a little scroll open in his hand. And │                  │
│ he set his right foot on the sea, and his     │ he set his right foot on the sea, and his    │                  │
│ left foot on the land, and called out with a  │ left foot on the land, and called out with a │                  │
│ loud voice, like a lion roaring. When he      │ loud voice, like a lion roaring. When he     │                  │
│ called out, the seven thunders sounded. And   │ called out, the seven thunders sounded. And  │                  │
│ when the seven thunders had sounded, I was    │ when the seven thunders had sounded, I was   │                  │
│ about to write, but I heard a voice from      │ about to write, but I heard a voice from     │                  │
│ heaven saying, “Seal up what the seven        │ heaven saying, “Seal up what the seven       │                  │
│ thunders have said, and do not write it       │ thunders have said, and do not write it      │                  │
│ down.” And the angel whom I saw standing on   │ down.” And the angel whom I saw standing on  │                  │
│ the sea and on the land raised his right hand │ the sea and on the land raised his right     │                  │
│ to heaven This is a quote from the Book of    │ hand to heaven And he swore by him who lives │                  │
│ Revelation in the Bible. The passage          │ forever and ever, who created heaven and     │                  │
│ describes an angel with a rainbow over his    │ what is in it, the earth and what is in it,  │                  │
│ head, a face like the sun, and legs like      │ and the sea and what is in it, that there    │                  │
│ pillars of fire. The angel has a small scroll │ would be no more delay, but that in the days │                  │
│ in his hand and sets one foot on the sea and  │ of the sixth seal, there would be a great    │                  │
│ the other on the land. The angel then roars   │ earthquake, and the sun became black as      │                  │
│ like a lion and the seven thunders sound. The │ sackcloth made of goat hair, the whole moon  │                  │
│ voice from heaven tells the angel to seal up  │ became like blood, and the stars of the sky  │                  │
│ what the seven thunders have said and not to  │ fell to the earth as the fig tree sheds its  │                  │
│ write it down.                                │ winter fruit when shaken by the wind. The    │                  │
│                                               │ sky vanished like a scroll that is being     │                  │
│                                               │ rolled up, and every mountain and island was │                  │
│                                               │ removed from its place. Then the kings of    │                  │
│                                               │ the earth and the great ones and the         │                  │
│                                               │ generals and the rich and the powerful, and  │                  │
│                                               │ everyo

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Then I saw another mighty angel coming down   │ Then I saw another mighty angel coming down  │ [6.1816, 5.9677] │
│ from heaven, wrapped in a cloud, with a       │ from heaven, wrapped in a cloud, with a      │                  │
│ rainbow over his head, and his face was like  │ rainbow over his head, and his face was like │                  │
│ the sun, and his legs like pillars of fire.   │ the sun, and his legs like pillars of fire.  │                  │
│ He had a little scroll open in his hand. And  │ He had a little scroll open in his hand. And │                  │
│ he set his right foot on the sea, and his     │ he set his right foot on the sea, and his    │                  │
│ left foot on the land, and called out with a  │ left foot on the land, and called out with a │                  │
│ loud voice, like a lion roaring. When he      │ loud voice, like a lion roaring. When he     │                  │
│ called out, the seven thunders sounded. And   │ called out, the seven thunders sounded. And  │                  │
│ when the seven thunders had sounded, I was    │ when the seven thunders had sounded, I was   │                  │
│ about to write, but I heard a voice from      │ about to write, but I heard a voice from     │                  │
│ heaven saying, “Seal up what the seven        │ heaven saying, “Seal up what the seven       │                  │
│ thunders have said, and do not write it       │ thunders have said, and do not write it      │                  │
│ down.” And the angel whom I saw standing on   │ down.” And the angel whom I saw standing on  │                  │
│ the sea and on the land raised his right hand │ the sea and on the land raised his right     │                  │
│ to heaven This is a quote from the Book of    │ hand to heaven And he swore by him who lives │                  │
│ Revelation in the Bible. The passage          │ forever and ever, who created heaven and     │                  │
│ describes an angel with a rainbow over his    │ what is in it, the earth and what is in it,  │                  │
│ head, a face like the sun, and legs like      │ and the sea and what is in it, that there    │                  │
│ pillars of fire. The angel has a small scroll │ would be no more delay, but that in the days │                  │
│ in his hand and sets one foot on the sea and  │ of the sixth seal, there would be a great    │                  │
│ the other on the land. The angel then roars   │ earthquake, and the sun became black as      │                  │
│ like a lion and the seven thunders sound. The │ sackcloth made of goat hair, the whole moon  │                  │
│ voice from heaven tells the angel to seal up  │ became like blood, and the stars of the sky  │                  │
│ what the seven thunders have said and not to  │ fell to the earth as the fig tree sheds its  │                  │
│ write it down.                                │ winter fruit when shaken by the wind. The    │                  │
│                                               │ sky vanished like a scroll that is being     │                  │
│                                               │ rolled up, and every mountain and island was │                  │
│                                               │ removed from its place. Then the kings of    │                  │
│                                               │ the earth and the great ones and the         │                  │
│                                               │ generals and the rich and the powerful, and  │                  │
│                                               │ everyo

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Then I saw another mighty angel coming down   │ Then I saw another mighty angel coming down  │ [6.1301, 5.8843] │
│ from heaven, wrapped in a cloud, with a       │ from heaven, wrapped in a cloud, with a      │                  │
│ rainbow over his head, and his face was like  │ rainbow over his head, and his face was like │                  │
│ the sun, and his legs like pillars of fire.   │ the sun, and his legs like pillars of fire.  │                  │
│ He had a little scroll open in his hand. And  │ He had a little scroll open in his hand. And │                  │
│ he set his right foot on the sea, and his     │ he set his right foot on the sea, and his    │                  │
│ left foot on the land, and called out with a  │ left foot on the land, and called out with a │                  │
│ loud voice, like a lion roaring. When he      │ loud voice, like a lion roaring. When he     │                  │
│ called out, the seven thunders sounded. And   │ called out, the seven thunders sounded. And  │                  │
│ when the seven thunders had sounded, I was    │ when the seven thunders had sounded, I was   │                  │
│ about to write, but I heard a voice from      │ about to write, but I heard a voice from     │                  │
│ heaven saying, “Seal up what the seven        │ heaven saying, “Seal up what the seven       │                  │
│ thunders have said, and do not write it       │ thunders have said, and do not write it      │                  │
│ down.” And the angel whom I saw standing on   │ down.” And the angel whom I saw standing on  │                  │
│ the sea and on the land raised his right hand │ the sea and on the land raised his right     │                  │
│ to heaven This is a quote from the Book of    │ hand to heaven And he swore by him who lives │                  │
│ Revelation in the Bible. The passage          │ forever and ever, who created heaven and     │                  │
│ describes an angel with a rainbow over his    │ what is in it, the earth and what is in it,  │                  │
│ head, a face like the sun, and legs like      │ and the sea and what is in it, that there    │                  │
│ pillars of fire. The angel has a small scroll │ would be no more delay, but that in the days │                  │
│ in his hand and sets one foot on the sea and  │ of the sixth seal, there would be a great    │                  │
│ the other on the land. The angel then roars   │ earthquake, and the sun became black as      │                  │
│ like a lion and the seven thunders sound. The │ sackcloth made of goat hair, the whole moon  │                  │
│ voice from heaven tells the angel to seal up  │ became like blood, and the stars of the sky  │                  │
│ what the seven thunders have said and not to  │ fell to the earth as the fig tree sheds its  │                  │
│ write it down.                                │ winter fruit when shaken by the wind. The    │                  │
│                                               │ sky vanished like a scroll that is being     │                  │
│                                               │ rolled up, and every mountain and island was │                  │
│                                               │ removed from its place. Then the kings of    │                  │
│                                               │ the earth and the great ones and the         │                  │
│                                               │ generals and the rich and the powerful, and  │                  │
│                                               │ everyo

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Then I saw another mighty angel coming down   │ Then I saw another mighty angel coming down  │ [6.0523, 5.837]  │
│ from heaven, wrapped in a cloud, with a       │ from heaven, wrapped in a cloud, with a      │                  │
│ rainbow over his head, and his face was like  │ rainbow over his head, and his face was like │                  │
│ the sun, and his legs like pillars of fire.   │ the sun, and his legs like pillars of fire.  │                  │
│ He had a little scroll open in his hand. And  │ He had a little scroll open in his hand. And │                  │
│ he set his right foot on the sea, and his     │ he set his right foot on the sea, and his    │                  │
│ left foot on the land, and called out with a  │ left foot on the land, and called out with a │                  │
│ loud voice, like a lion roaring. When he      │ loud voice, like a lion roaring. When he     │                  │
│ called out, the seven thunders sounded. And   │ called out, the seven thunders sounded. And  │                  │
│ when the seven thunders had sounded, I was    │ when the seven thunders had sounded, I was   │                  │
│ about to write, but I heard a voice from      │ about to write, but I heard a voice from     │                  │
│ heaven saying, “Seal up what the seven        │ heaven saying, “Seal up what the seven       │                  │
│ thunders have said, and do not write it       │ thunders have said, and do not write it      │                  │
│ down.” And the angel whom I saw standing on   │ down.” And the angel whom I saw standing on  │                  │
│ the sea and on the land raised his right hand │ the sea and on the land raised his right     │                  │
│ to heaven This is a quote from the Book of    │ hand to heaven And he swore by him who lives │                  │
│ Revelation in the Bible. The passage          │ forever and ever, who created heaven and     │                  │
│ describes an angel with a rainbow over his    │ what is in it, the earth and what is in it,  │                  │
│ head, a face like the sun, and legs like      │ and the sea and what is in it, that there    │                  │
│ pillars of fire. The angel has a small scroll │ would be no more delay, but that in the days │                  │
│ in his hand and sets one foot on the sea and  │ of the sixth seal, there would be a great    │                  │
│ the other on the land. The angel then roars   │ earthquake, and the sun became black as      │                  │
│ like a lion and the seven thunders sound. The │ sackcloth made of goat hair, the whole moon  │                  │
│ voice from heaven tells the angel to seal up  │ became like blood, and the stars of the sky  │                  │
│ what the seven thunders have said and not to  │ fell to the earth as the fig tree sheds its  │                  │
│ write it down.                                │ winter fruit when shaken by the wind. The    │                  │
│                                               │ sky vanished like a scroll that is being     │                  │
│                                               │ rolled up, and every mountain and island was │                  │
│                                               │ removed from its place. Then the kings of    │                  │
│                                               │ the earth and the great ones and the         │                  │
│                                               │ generals and the rich and the powerful, and  │                  │
│                                               │ everyo

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Then I saw another mighty angel coming down   │ Then I saw another mighty angel coming down  │ [6.003, 5.8225]  │
│ from heaven, wrapped in a cloud, with a       │ from heaven, wrapped in a cloud, with a      │                  │
│ rainbow over his head, and his face was like  │ rainbow over his head, and his face was like │                  │
│ the sun, and his legs like pillars of fire.   │ the sun, and his legs like pillars of fire.  │                  │
│ He had a little scroll open in his hand. And  │ He had a little scroll open in his hand. And │                  │
│ he set his right foot on the sea, and his     │ he set his right foot on the sea, and his    │                  │
│ left foot on the land, and called out with a  │ left foot on the land, and called out with a │                  │
│ loud voice, like a lion roaring. When he      │ loud voice, like a lion roaring. When he     │                  │
│ called out, the seven thunders sounded. And   │ called out, the seven thunders sounded. And  │                  │
│ when the seven thunders had sounded, I was    │ when the seven thunders had sounded, I was   │                  │
│ about to write, but I heard a voice from      │ about to write, but I heard a voice from     │                  │
│ heaven saying, “Seal up what the seven        │ heaven saying, “Seal up what the seven       │                  │
│ thunders have said, and do not write it       │ thunders have said, and do not write it      │                  │
│ down.” And the angel whom I saw standing on   │ down.” And the angel whom I saw standing on  │                  │
│ the sea and on the land raised his right hand │ the sea and on the land raised his right     │                  │
│ to heaven This is a quote from the Book of    │ hand to heaven And he swore by him who lives │                  │
│ Revelation in the Bible. The passage          │ forever and ever, who created heaven and     │                  │
│ describes an angel with a rainbow over his    │ what is in it, the earth and what is in it,  │                  │
│ head, a face like the sun, and legs like      │ and the sea and what is in it, that there    │                  │
│ pillars of fire. The angel has a small scroll │ would be no more delay, but that in the days │                  │
│ in his hand and sets one foot on the sea and  │ of the sixth seal, there would be a great    │                  │
│ the other on the land. The angel then roars   │ earthquake, and the sun became black as      │                  │
│ like a lion and the seven thunders sound. The │ sackcloth made of goat hair, the whole moon  │                  │
│ voice from heaven tells the angel to seal up  │ became like blood, and the stars of the sky  │                  │
│ what the seven thunders have said and not to  │ fell to the earth as the fig tree sheds its  │                  │
│ write it down.                                │ winter fruit when shaken by the wind. The    │                  │
│                                               │ sky vanished like a scroll that is being     │                  │
│                                               │ rolled up, and every mountain and island was │                  │
│                                               │ removed from its place. Then the kings of    │                  │
│                                               │ the earth and the great ones and the         │                  │
│                                               │ generals and the rich and the powerful, and  │                  │
│                                               │ everyo

TrainOutput(global_step=324, training_loss=0.8692544065875771, metrics={'train_runtime': 1927.9567, 'train_samples_per_second': 5.409, 'train_steps_per_second': 0.168, 'total_flos': 0.0, 'train_loss': 0.8692544065875771, 'epoch': 2.994246260069045})

### Save the model

In [22]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

login(os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [23]:
repo_name = "MilyaShams/SmolLM2-135M-Instruct-Reward-probabilistic"

merged_model = trainer.model.merge_and_unload()
merged_model = merged_model.half()

merged_model.push_to_hub(repo_id=repo_name, tags=["LoRa"], torch_dtype=torch.float16)
tokenizer.push_to_hub(repo_id=repo_name)

model.safetensors: 100%|██████████| 269M/269M [00:42<00:00, 6.35MB/s] 


CommitInfo(commit_url='https://huggingface.co/MilyaShams/SmolLM2-135M-Instruct-Reward-with-discrete-scores/commit/9896c3347a49daa45a9b01b3f798dc0ac0b3b145', commit_message='Upload tokenizer', commit_description='', oid='9896c3347a49daa45a9b01b3f798dc0ac0b3b145', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MilyaShams/SmolLM2-135M-Instruct-Reward-with-discrete-scores', endpoint='https://huggingface.co', repo_type='model', repo_id='MilyaShams/SmolLM2-135M-Instruct-Reward-with-discrete-scores'), pr_revision=None, pr_num=None)